In [1]:
import os
import sys

from tqdm.notebook import tqdm
from sentiment_utils import clean_mentions, get_batches, apply_sentiment_analysis, convert_to_list, update_sentiment_scores

sys.path.append(os.path.join(os.path.dirname(os.getcwd()), "_0_Constants_and_Utils"))

from database_utils import get_dataframe_from_query, form_connection_params


c:\My programs\Python 3.12\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

All the layers of TFRobertaForSequenceClassification were initialized from the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaForSequenceClassification for predictions without further training.


In [2]:
local = True
connection_params = form_connection_params(local, True)
query = """
SELECT tweet_id, full_text
FROM Tweets
WHERE sentiment_score IS NULL;
"""
batch_size = 10_000

In [3]:
test_data = get_dataframe_from_query(query, connection_params, local, index_col="tweet_id")

In [4]:
# Apply the cleaning function to the DataFrame
test_data['cleaned_text'] = test_data['full_text'].apply(clean_mentions)

In [5]:
test_data

,full_text,cleaned_text
tweet_id,,
1131172858951024641,La ruta de easyJet entre Londres y Menorca tra...,La ruta de easyJet entre Londres y Menorca tra...
1130922003702177800,@goody_tracy Here’s a list of some of @JonesDa...,Here’s a list of some of clients They should ...
1131172864147808257,RT @bttr_as1: @goody_tracy Here’s a list of so...,Here’s a list of some of clients They should ...
1131172867985485824,@British_Airways,
1131030279278063616,Nice change by @AmericanAir. Bikes now pay sta...,Nice change by Bikes now pay standard checked...
...,...,...
1244696703690772485,RT @jfergo86: Me parece a mí o el avión es más...,Me parece a mí o el avión es más grande que el...
1244696708983984131,Today’s random pic of the day is the one of Vo...,Today’s random pic of the day is the one of Vo...
1244696710447800320,RT @SchipholWatch: @spbverhagen @markduursma @...,Nog niet aan de orde? Als in er is nog geen st...


In [6]:
data_batches = get_batches(test_data[["cleaned_text"]], batch_size)

In [7]:
for batch in tqdm(data_batches, desc="Updating sentiment_score: "):
    # feel free to change the 
    df_sentiment = apply_sentiment_analysis(batch, "cleaned_text", 128, 10)
    update_sentiment_scores(convert_to_list(df_sentiment), connection_params, local)

Updating sentiment_score:   0%|          | 0/6149 [00:00<?, ?it/s]

KeyboardInterrupt: 